In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-07 02:17:45.244398: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=1000)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_2_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_2_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dropout (Dropout)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 500)               3600500   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                       

Epoch 24/1000
7/7 [==============================] - 0s 21ms/step - loss: 10.5027 - mean_squared_error: 10.5027 - rmse: 3.2408 - mean_absolute_error: 2.5740 - val_loss: 18.2628 - val_mean_squared_error: 18.2628 - val_rmse: 4.2735 - val_mean_absolute_error: 3.1619 - lr: 5.0000e-04
Epoch 25/1000
7/7 [==============================] - 0s 20ms/step - loss: 9.6671 - mean_squared_error: 9.6671 - rmse: 3.1092 - mean_absolute_error: 2.5033 - val_loss: 17.3974 - val_mean_squared_error: 17.3974 - val_rmse: 4.1710 - val_mean_absolute_error: 3.0714 - lr: 5.0000e-04
Epoch 26/1000
7/7 [==============================] - 0s 20ms/step - loss: 10.1128 - mean_squared_error: 10.1128 - rmse: 3.1801 - mean_absolute_error: 2.5031 - val_loss: 16.0005 - val_mean_squared_error: 16.0005 - val_rmse: 4.0001 - val_mean_absolute_error: 2.9304 - lr: 5.0000e-04
Epoch 27/1000
7/7 [==============================] - 0s 20ms/step - loss: 9.9271 - mean_squared_error: 9.9271 - rmse: 3.1507 - mean_absolute_error: 2.5257 - va

Epoch 53/1000
7/7 [==============================] - 0s 21ms/step - loss: 8.5958 - mean_squared_error: 8.5958 - rmse: 2.9319 - mean_absolute_error: 2.3158 - val_loss: 14.6632 - val_mean_squared_error: 14.6632 - val_rmse: 3.8293 - val_mean_absolute_error: 2.8441 - lr: 2.5000e-04
Epoch 54/1000
7/7 [==============================] - 0s 21ms/step - loss: 7.9732 - mean_squared_error: 7.9732 - rmse: 2.8237 - mean_absolute_error: 2.2446 - val_loss: 14.5353 - val_mean_squared_error: 14.5353 - val_rmse: 3.8125 - val_mean_absolute_error: 2.8273 - lr: 2.5000e-04
Epoch 55/1000
7/7 [==============================] - 0s 21ms/step - loss: 7.7987 - mean_squared_error: 7.7987 - rmse: 2.7926 - mean_absolute_error: 2.1807 - val_loss: 13.9438 - val_mean_squared_error: 13.9438 - val_rmse: 3.7341 - val_mean_absolute_error: 2.7743 - lr: 2.5000e-04
Epoch 56/1000
7/7 [==============================] - 0s 21ms/step - loss: 8.3922 - mean_squared_error: 8.3922 - rmse: 2.8969 - mean_absolute_error: 2.2856 - val_lo

Epoch 82/1000
7/7 [==============================] - 0s 23ms/step - loss: 6.7564 - mean_squared_error: 6.7564 - rmse: 2.5993 - mean_absolute_error: 2.0478 - val_loss: 10.5740 - val_mean_squared_error: 10.5740 - val_rmse: 3.2518 - val_mean_absolute_error: 2.4275 - lr: 1.2500e-04
Epoch 83/1000
7/7 [==============================] - 0s 24ms/step - loss: 7.0061 - mean_squared_error: 7.0061 - rmse: 2.6469 - mean_absolute_error: 2.0483 - val_loss: 10.2377 - val_mean_squared_error: 10.2377 - val_rmse: 3.1996 - val_mean_absolute_error: 2.3923 - lr: 1.2500e-04
Epoch 84/1000
7/7 [==============================] - 0s 21ms/step - loss: 7.3920 - mean_squared_error: 7.3920 - rmse: 2.7188 - mean_absolute_error: 2.0915 - val_loss: 11.3827 - val_mean_squared_error: 11.3827 - val_rmse: 3.3738 - val_mean_absolute_error: 2.5175 - lr: 1.2500e-04
Epoch 85/1000
7/7 [==============================] - 0s 29ms/step - loss: 6.5775 - mean_squared_error: 6.5775 - rmse: 2.5647 - mean_absolute_error: 1.9993 - val_lo

7/7 [==============================] - 0s 21ms/step - loss: 6.1085 - mean_squared_error: 6.1085 - rmse: 2.4715 - mean_absolute_error: 1.9012 - val_loss: 10.5957 - val_mean_squared_error: 10.5957 - val_rmse: 3.2551 - val_mean_absolute_error: 2.4236 - lr: 1.2500e-04
Epoch 112/1000
7/7 [==============================] - 0s 22ms/step - loss: 6.7951 - mean_squared_error: 6.7951 - rmse: 2.6067 - mean_absolute_error: 2.0396 - val_loss: 10.2412 - val_mean_squared_error: 10.2412 - val_rmse: 3.2002 - val_mean_absolute_error: 2.3784 - lr: 1.2500e-04
Epoch 113/1000
7/7 [==============================] - 0s 20ms/step - loss: 7.1046 - mean_squared_error: 7.1046 - rmse: 2.6654 - mean_absolute_error: 2.0588 - val_loss: 9.9412 - val_mean_squared_error: 9.9412 - val_rmse: 3.1530 - val_mean_absolute_error: 2.3385 - lr: 1.2500e-04
Epoch 114/1000
7/7 [==============================] - 0s 27ms/step - loss: 6.4749 - mean_squared_error: 6.4749 - rmse: 2.5446 - mean_absolute_error: 1.9995 - val_loss: 8.4922 - 

...

Epoch 30/1000
7/7 [==============================] - 0s 33ms/step - loss: 9.9329 - mean_squared_error: 9.9329 - rmse: 3.1517 - mean_absolute_error: 2.5209 - val_loss: 13.5433 - val_mean_squared_error: 13.5433 - val_rmse: 3.6801 - val_mean_absolute_error: 2.7318 - lr: 5.0000e-04
Epoch 31/1000
7/7 [==============================] - 0s 29ms/step - loss: 10.0694 - mean_squared_error: 10.0694 - rmse: 3.1732 - mean_absolute_error: 2.5296 - val_loss: 11.7016 - val_mean_squared_error: 11.7016 - val_rmse: 3.4208 - val_mean_absolute_error: 2.5243 - lr: 5.0000e-04
Epoch 32/1000
7/7 [==============================] - 0s 29ms/step - loss: 10.6262 - mean_squared_error: 10.6262 - rmse: 3.2598 - mean_absolute_error: 2.5859 - val_loss: 11.1771 - val_mean_squared_error: 11.1771 - val_rmse: 3.3432 - val_mean_absolute_error: 2.4749 - lr: 5.0000e-04
Epoch 33/1000
7/7 [==============================] - 0s 32ms/step - loss: 10.4500 - mean_squared_error: 10.4500 - rmse: 3.2327 - mean_absolute_error: 2.5422 - 

7/7 [==============================] - 0s 30ms/step - loss: 7.6137 - mean_squared_error: 7.6137 - rmse: 2.7593 - mean_absolute_error: 2.0992 - val_loss: 8.0190 - val_mean_squared_error: 8.0190 - val_rmse: 2.8318 - val_mean_absolute_error: 2.0730 - lr: 5.0000e-04
Epoch 60/1000
7/7 [==============================] - 0s 28ms/step - loss: 7.2282 - mean_squared_error: 7.2282 - rmse: 2.6885 - mean_absolute_error: 2.0981 - val_loss: 11.8792 - val_mean_squared_error: 11.8792 - val_rmse: 3.4466 - val_mean_absolute_error: 2.5696 - lr: 5.0000e-04
Epoch 61/1000
7/7 [==============================] - 0s 33ms/step - loss: 8.5980 - mean_squared_error: 8.5980 - rmse: 2.9322 - mean_absolute_error: 2.2642 - val_loss: 9.0005 - val_mean_squared_error: 9.0005 - val_rmse: 3.0001 - val_mean_absolute_error: 2.2145 - lr: 5.0000e-04
Epoch 62/1000
7/7 [==============================] - 0s 30ms/step - loss: 7.1454 - mean_squared_error: 7.1454 - rmse: 2.6731 - mean_absolute_error: 2.1022 - val_loss: 10.6472 - val_

Epoch 88/1000
7/7 [==============================] - 0s 33ms/step - loss: 6.6035 - mean_squared_error: 6.6035 - rmse: 2.5697 - mean_absolute_error: 1.9876 - val_loss: 10.9560 - val_mean_squared_error: 10.9560 - val_rmse: 3.3100 - val_mean_absolute_error: 2.4737 - lr: 2.5000e-04
Epoch 89/1000
7/7 [==============================] - 0s 36ms/step - loss: 6.2580 - mean_squared_error: 6.2580 - rmse: 2.5016 - mean_absolute_error: 1.9439 - val_loss: 11.2848 - val_mean_squared_error: 11.2848 - val_rmse: 3.3593 - val_mean_absolute_error: 2.5117 - lr: 2.5000e-04
Epoch 90/1000
7/7 [==============================] - ETA: 0s - loss: 6.7844 - mean_squared_error: 6.7844 - rmse: 2.6047 - mean_absolute_error: 2.0174
Epoch 00090: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
7/7 [==============================] - 0s 35ms/step - loss: 6.7844 - mean_squared_error: 6.7844 - rmse: 2.6047 - mean_absolute_error: 2.0174 - val_loss: 10.0507 - val_mean_squared_error: 10.0507 - val_rmse: 3.170

Epoch 116/1000
7/7 [==============================] - 0s 37ms/step - loss: 5.7744 - mean_squared_error: 5.7744 - rmse: 2.4030 - mean_absolute_error: 1.8794 - val_loss: 11.3148 - val_mean_squared_error: 11.3148 - val_rmse: 3.3638 - val_mean_absolute_error: 2.5397 - lr: 6.2500e-05
Epoch 117/1000
7/7 [==============================] - 0s 36ms/step - loss: 6.0411 - mean_squared_error: 6.0411 - rmse: 2.4579 - mean_absolute_error: 1.9087 - val_loss: 8.9605 - val_mean_squared_error: 8.9605 - val_rmse: 2.9934 - val_mean_absolute_error: 2.1983 - lr: 6.2500e-05
Epoch 118/1000
7/7 [==============================] - 0s 35ms/step - loss: 6.2197 - mean_squared_error: 6.2197 - rmse: 2.4939 - mean_absolute_error: 1.9480 - val_loss: 11.8917 - val_mean_squared_error: 11.8917 - val_rmse: 3.4484 - val_mean_absolute_error: 2.6193 - lr: 6.2500e-05
Epoch 119/1000
7/7 [==============================] - 0s 36ms/step - loss: 5.9404 - mean_squared_error: 5.9404 - rmse: 2.4373 - mean_absolute_error: 1.9182 - val_

Epoch 145/1000
7/7 [==============================] - 0s 34ms/step - loss: 5.9137 - mean_squared_error: 5.9137 - rmse: 2.4318 - mean_absolute_error: 1.8823 - val_loss: 10.7166 - val_mean_squared_error: 10.7166 - val_rmse: 3.2736 - val_mean_absolute_error: 2.4495 - lr: 3.1250e-05
Epoch 146/1000
7/7 [==============================] - 0s 26ms/step - loss: 6.1036 - mean_squared_error: 6.1036 - rmse: 2.4706 - mean_absolute_error: 1.9303 - val_loss: 10.0621 - val_mean_squared_error: 10.0621 - val_rmse: 3.1721 - val_mean_absolute_error: 2.3550 - lr: 3.1250e-05
Epoch 147/1000
7/7 [==============================] - 0s 42ms/step - loss: 6.1005 - mean_squared_error: 6.1005 - rmse: 2.4699 - mean_absolute_error: 1.9311 - val_loss: 10.4687 - val_mean_squared_error: 10.4687 - val_rmse: 3.2355 - val_mean_absolute_error: 2.4151 - lr: 3.1250e-05
Epoch 148/1000
7/7 [==============================] - 0s 34ms/step - loss: 6.1160 - mean_squared_error: 6.1160 - rmse: 2.4731 - mean_absolute_error: 1.9174 - va

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)